In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
gme = yf.Ticker("GME")

In [3]:
gme.info

{'zip': '76051',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 12000,
 'longBusinessSummary': 'GameStop Corp., a specialty retailer, provides games and entertainment products through its e-commerce properties and various stores in the United States, Canada, Australia, and Europe. The company sells new and pre-owned video game platforms; accessories, such as controllers, gaming headsets, virtual reality products, and memory cards; new and pre-owned video game software; and in-game digital currency, digital downloadable content, and full-game downloads, as well as network points cards, and prepaid digital and subscription cards. It also sells collectibles comprising licensed merchandise primarily related to the video game, television, and movie industries, as well as pop culture themes. The company operates its stores and e-commerce sites under the GameStop, EB Games, and Micromania brands; and collectibles stores under the Zing Pop Culture and ThinkGeek brand, as well as offers 

In [4]:
stock_df = gme.history(period='3y',interval='1d', start='2019-01-01', end=None)
stock_df.shape

(575, 7)

In [5]:
stock_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,12.041450,12.747496,12.012435,12.641105,2124200,0.0,0
2019-01-03,12.554058,12.824870,12.273574,12.544387,1750400,0.0,0
2019-01-04,13.569602,14.788255,13.250431,14.739896,11915700,0.0,0
2019-01-07,14.614162,15.291192,14.449740,14.972020,4718000,0.0,0
2019-01-08,14.991364,15.368567,14.652849,15.291192,3443800,0.0,0
...,...,...,...,...,...,...,...
2021-04-08,185.880005,185.880005,164.300003,170.259995,10047400,0.0,0
2021-04-09,169.699997,171.580002,153.000000,158.360001,9442000,0.0,0
2021-04-12,158.110001,163.899994,135.009995,141.089996,16683600,0.0,0


In [6]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [7]:
def buy_sell(row):
    if row['Close'] < row['NextClose']:
        val = 'buy'
    elif row['Close'] > row['NextClose']:
        val = 'sell'
    else:
        val = 'hold'
    return val


buy_sel_dict = {'buy': 1, 'sell': 0, 'hold': 2}

In [8]:
stock_df['RSI'] = computeRSI(stock_df.Close, 14)
stock_df['NextClose'] = stock_df.Close.shift(-1)
stock_df['BuySell'] = stock_df.apply(buy_sell, axis = 1).map(buy_sel_dict)
stock_df = stock_df.fillna(0)

In [9]:
stock_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI,NextClose,BuySell
Date,,,,,,,,,,
2019-01-02,12.041450,12.747496,12.012435,12.641105,2124200,0.0,0,0.0,12.544387,0
2019-01-03,12.554058,12.824870,12.273574,12.544387,1750400,0.0,0,0.0,14.739896,1
2019-01-04,13.569602,14.788255,13.250431,14.739896,11915700,0.0,0,0.0,14.972020,1
2019-01-07,14.614162,15.291192,14.449740,14.972020,4718000,0.0,0,0.0,15.291192,1
2019-01-08,14.991364,15.368567,14.652849,15.291192,3443800,0.0,0,0.0,15.455612,1


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, plot_confusion_matrix, classification_report

In [11]:
X = stock_df.iloc[:,: -2]
y = stock_df.BuySell
X_train, X_test, y_train, y_test = train_test_split(X, y)

RF = RandomForestClassifier(n_estimators=1000, max_depth=2, random_state=0)
RF.fit(X_train, y_train)

print('Accuracy:', accuracy_score(y_test, RF.predict(X_test)))
print('Confusion Matrix:', confusion_matrix(y_test, RF.predict(X_test)))

NameError: name 'train_test_split' is not defined

In [ ]:
plot_confusion_matrix(RF, X_test, y_test)

In [ ]:
print(classification_report(y_test, RF.predict(X_test)))

In [ ]:
stock_df.BuySell.value_counts()

In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
X = stock_df.drop(labels='NextClose', axis = 1)
y = stock_df.NextClose

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
regr = LinearRegression(fit_intercept=False)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_test.index, y_test, color = 'r')
plt.scatter(y_test.index, y_pred, color = 'b')

In [ ]:
print('r^2 error:', round(r2_score(y_test, y_pred), 3))
print('MAE:', round(mean_absolute_error(y_test, y_pred), 3))
print('Mean % Error:', round(mean_absolute_percentage_error(y_test, y_pred), 3))

In [ ]:
stock_df_backtest = gme.history(period='1y', interval='1d', start='2018-01-01', end = '2018-12-25')
stock_df_backtest['RSI'] = computeRSI(stock_df_backtest.Close, 14)
stock_df_backtest['NextClose'] = stock_df_backtest.Close.shift(-1)
stock_df_backtest['BuySell'] = stock_df_backtest.apply(buy_sell, axis = 1).map(buy_sel_dict)
stock_df_backtest = stock_df_backtest.fillna(0)[:-1]

In [ ]:
y_val = stock_df_backtest.NextClose
X_val = stock_df_backtest.drop(labels='NextClose', axis = 1)
y_val_pred = regr.predict(X_val)

plt.figure(figsize=(15,10))
plt.plot(y_val.index, y_val, color = 'r')
plt.plot(y_val.index, y_val_pred, color = 'b')

In [ ]:
print('r^2 error:', round(r2_score(y_val, y_val_pred), 3))
print('MAE:', round(mean_absolute_error(y_val, y_val_pred), 3))
print('Mean % Error:', round(mean_absolute_percentage_error(y_val, y_val_pred), 3))

In [ ]:
import pmdarima as pm
from pmdarima.model_selection import train_test_split

In [ ]:
train, test = train_test_split(stock_df.Close, train_size=0.2)

In [ ]:


# Fit your model
model = pm.auto_arima(train, seasonal=True, m=12)

# make your forecasts
forecasts = model.predict(test.shape[0])  # predict N steps into the future

# Visualize the forecasts (blue=train, green=forecasts)
# x = np.arange(stock_df.Close.shape[0])


In [ ]:
x = stock_df.index
plt.figure(figsize=(10,10))
plt.plot(x[:150], train, c='blue')
plt.plot(x[:len(test)], test, c='blue')
plt.plot(x[150:], forecasts, c='green')
plt.show()

In [ ]:
stock_df.Close.plot()

In [ ]:
stock_df.ewm()